<a href="https://colab.research.google.com/github/NeelKanwal/BERTOLOGY-Based-Extractive-Summarization-for-Clinial-Notes/blob/master/BertViz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BertViz
(Tool Designed by: Jessie Vig  https://github.com/jessevig/bertviz.git)

Attention has been very important recently in Natural Language Processing. Self attention described how model perceives importa word embeddings. After advent of transformers we needed manual tools to underastand this mechanism.

Now Wait is OVER. 

BERTViz is a visualization tool extended from huggingface transform and tensor2tensor visualization. In this notebook, I will present some details of how to make different models and how to understand them for nature of task.

Lets start by installing necessary modules.


In [6]:
!pip install regex
!pip install transformers

In [8]:
!git clone https://github.com/jessevig/bertviz.git
!cd bertviz

Cloning into 'bertviz'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 1074 (delta 32), reused 43 (delta 19), pack-reused 1013
Receiving objects: 100% (1074/1074), 100.13 MiB | 23.98 MiB/s, done.
Resolving deltas: 100% (681/681), done.


# 1.Neuron View

In [9]:
import sys
!test -d bertviz_repo && echo "FYI: bertviz_repo directory already exists, to pull latest version uncomment this line: !rm -r bertviz_repo"
# !rm -r bertviz_repo # Uncomment if you need a clean pull from repo
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']



def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/5.7.0/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))


FYI: bertviz_repo directory already exists, to pull latest version uncomment this line: !rm -r bertviz_repo


# Transformers has a unified API
for 10 transformer architectures and 30 pretrained weights.
#          Model          | Tokenizer          | Pretrained weights shortcut


```
MODELS = [(BertModel,       BertTokenizer,       'bert-base-uncased'),
          (OpenAIGPTModel,  OpenAIGPTTokenizer,  'openai-gpt'),
          (GPT2Model,       GPT2Tokenizer,       'gpt2'),
          (CTRLModel,       CTRLTokenizer,       'ctrl'),
          (TransfoXLModel,  TransfoXLTokenizer,  'transfo-xl-wt103'),
          (XLNetModel,      XLNetTokenizer,      'xlnet-base-cased'),
          (XLMModel,        XLMTokenizer,        'xlm-mlm-enfr-1024'),
          (DistilBertModel, DistilBertTokenizer, 'distilbert-base-uncased'),
          (RobertaModel,    RobertaTokenizer,    'roberta-base'),
          (XLMRobertaModel, XLMRobertaTokenizer, 'xlm-roberta-base'),
         ]
```




In [0]:
from bertviz.transformers_neuron_view import BertModel, BertTokenizer
model_version = 'bert-base-uncased'
model = BertModel.from_pretrained(model_version)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=True)

In [19]:
from bertviz.neuron_view import show
sentence="She had a chest cta"
call_html()
show(model, 'bert', tokenizer, sentence)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#2.Model View

In [16]:
from bertviz import model_view

def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/5.7.0/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))
   
call_html()
model_view(attentions, tokens)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#3. Head View

Using available models from huggingface/transformer and putting it on evaluation. We use encode plues to encode sentence into (768,) embeddings and masks. 
Setting special tokens to TRUE and return tensors to pt to get pytorch tensors for whole sentence plus [CLS] and [SEP]

In [20]:
from transformers import BertModel, BertTokenizer
from bertviz import head_view
import torch

def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
         '''))

pretrained_weights = 'bert-base-uncased'
model = BertModel.from_pretrained(pretrained_weights,output_attentions=True)
model.eval()
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
input_ = tokenizer.encode_plus(sentence,add_special_tokens=True,return_tensors='pt')
input_

{'input_ids': tensor([[  101,  2016,  2018,  1037,  3108, 14931,  2050,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

For multiple sentnces we perform padding to make all tensors to equal size.

In [21]:
# tensor_text = pad_sequence([item['input_ids'].squeeze(0) for item in input_text], batch_first=True)
# tensor_mask = pad_sequence([item['attention_mask'].squeeze(0) for item in input_text], batch_first=True)
# tensor_ids = pad_sequence([item['token_type_ids'].squeeze(0) for item in input_text], batch_first=True)
if torch.cuda.is_available():
  print("GPU is availabel")
  model, tensor_text, token_type, attention_mask = model.cuda(), input_['input_ids'].cuda(), input_['token_type_ids'].cuda(), input_['attention_mask'].cuda()
  output, loss, attentions = model(tensor_text, token_type_ids = token_type, attention_mask = attention_mask)  
else:
  print("CPU is available")
  output, loss, attentions = model(input_['input_ids'], token_type_ids = input_['token_type_ids'], attention_mask = input_['attention_mask']) 

CPU is available


In [22]:
tokens = tokenizer.convert_ids_to_tokens(input_['input_ids'][0])

call_html()
head_view(attentions, tokens)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>